# Step 0

## - Click on the menu to the right

## - "Insert Project Token"

## - Use the downward arrow icon on the top menu to move the cell down and begin running the notebook

# Load Data into Milvus for RAG


# 1. Set up the environment

## Install libraries

we need to install the pymilvus package to the watsonx.ai Python environment. 

In [4]:
!pip install grpcio==1.60.0 
!pip install pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 36.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.54.3
    Uninstalling grpcio-1.54.3:
      Successfully uninstalled grpcio-1.54.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 35.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.2 MB/s eta 0:00:00


# !!RESTART THE KERNAL AFTER pymilvus install!! 

Certain dependencies need to be persisted. Restarting the kernal allows this to occur

In [1]:
!pip install ipython-sql==0.4.1
!pip install sqlalchemy==1.4.46
!pip install sqlalchemy==1.4.46 "pyhive[presto]"
!pip install python-dotenv
!pip install sentence_transformers
!pip install langchain-community

# clean up the libraries not required

  Preparing metadata (setup.py) ... done
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13695 sha256=e5b18e639fc22ef00a49ba74df2c03b0c30894686101ae30b90c763909c13763
  Stored in directory: /home/spark/shared/.cache/pip/wheels/b1/e4/65/051f9bd54c89d377db5ca942a672439c36cbf5ceff8e8902e0
Successfully built prettytable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.25
    Uninstalling SQLAlchemy-2.0.25:
      Successfully uninstalled SQLAlchemy-2.0.25
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyhive: filename=PyHive-0.7.0-py3-none-any.whl size=53872 sha256=c7ceb9f38c4086691a88a977f3ec62e1b7d41b6ecf89a8580e26b06d30b173b6
  Stored in directory: /home/spark/shared/.cache/pip/wheels/11/32/63/d1d379f01c15d6488b22ed89d257b613494e4595ed9b9c7f1c

# Step 1: Document Ingestion

## Load the pdf version of the watsonx.data documentation

Load the pdf version as an asset in the project using Spark

In [10]:
import requests
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder \
    .appName("Download watsonx.data PDF documentation") \
    .getOrCreate()

def download_pdf(url, local_path):
    """Download the PDF from a URL and save it locally"""
    response = requests.get(url)
    if response.status_code == 200:
        with open(local_path, 'wb') as file:
            file.write(response.content)
        print(f"PDF downloaded successfully to {local_path}")
    else:
        print(f"Failed to download PDF. Status code: {response.status_code}")

pdf_url = "https://www.ibm.com/support/pages/system/files/inline-files/IBM%20watsonx.data%20version%202.0.3.pdf"  
local_file_path = "wxd_doc_pdf.pdf"  

download_pdf(pdf_url, local_file_path)

spark.stop()


PDF downloaded successfully to wxd_doc_pdf.pdf


## Code to extract the text from the pdf for embeddings

In [13]:

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader

asset_li=wslib.assets.list_assets("data_asset")
print(asset_li)

wslib.download_file("wxd_doc_pdf")
# loaders = [
#     DirectoryLoader(directory, glob="**/*.html",show_progress=True),
#     DirectoryLoader(directory, glob="**/*.pdf",show_progress=True, loader_cls=PyPDFLoader),
#     DirectoryLoader(directory, glob="**/*.pptx",show_progress=True),
#     DirectoryLoader(directory, glob="**/*.docx",show_progress=True)
# ]


# documents=[]
# for loader in loaders:
#     data =loader.load()
#     documents.extend(data)
import fitz # PyMuPDF

doc = fitz.open("wxd_doc_pdf")

pdf_text = ""

for page in doc:
    pdf_text += page.get_text()

# print(pdf_text)

[{'name': 'wxd_doc_pdf', 'description': None, 'asset_id': '1377b784-7528-4933-ade1-096dafbf2d7a', 'asset_type': 'data_asset', 'tags': None}]


ModuleNotFoundError: No module named 'fitz'

# Step 2: Document Chunking

To manage the large texts, we can divide the data into manageble chunks by logical units such as paragraphs, sentences, or fixed token lengths

### Option 1: Chunking by Paragraphs

In [ ]:
def chunk_by_paragraphs(text):
    paragraphs = text.split("\n\n") # Assuming paragraphs are separated by two newlines
    return [p.strip() for p in paragraphs if p.strip()]

chunks = chunk_by_paragraphs(pdf_text)

print(chunks[:5]) # Preview of the first 5 chunks

### Option 2: Chunking by Sentences

If the document structure is more fluid and paragraphs are not clearly defined, you could break the text into sentences. You can use _nltk_ or a similar library for sentence tokennization

In [ ]:
import nltk

nltk.download('punkt')

def chunk_by_sentence(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

chunks = chunk_by_sentence(pdf_text)

print(chunks[:5]) # Preview of the first 5 chunks

### Option 3: Chunking by Token length

For more control over the chunk size, you can spilt the text into chunks of a fixed number of tokens

In [ ]:
def chunk_by_tokens(text, max_tokens=512):
    tokens = text.split() # Tokenize the whitespace
    chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [' '.join(chunk) for chunk in chunks]

chunks = chunk_by_tokens(pdf_text)

print(chunks[:5]) # Preview of the first 5 chunks

# Step 3: Embedding Generation

In [ ]:
wslib.list_connections

In [ ]:
# note if you named your Milvus connection something other than "Milvus Connection" Please replace the name below

milvus_credentials = wslib.get_connection("Milvus Connection")

# replace the milvus connection asset in the project

In [ ]:
#milvus_credentials

In [ ]:
from pymilvus import(
    Milvus,
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema,
)


url = milvus_credentials['host']
port = milvus_credentials['port']
apikey = milvus_credentials['password']
apiuser = 'ibmlhapikey'


connections.connect(alias="default", 
                    host=url, 
                    port=port, 
                    user=apiuser, 
                    password=apikey, 
                    secure=True)

In [ ]:
# Create a new collection
collection_description = 'wxd docs pdf'
collection_name = 'wxd_documentation'

In [ ]:
# Create collection - define fields + schema

fields = [
    FieldSchema(name="document_id", dtype=DataType.INT64), # Document Id
    FieldSchema(name="chunk_id",  dtype=DataType.INT64), # Chunk Id
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384), # embedding dimension
]

# Create a schema
schema = CollectionSchema(fields, collection_description)

# Create a collection
collection = Collection(collection_name, schema)

# Create index
index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
}

collection.create_index(field_name="vector", index_params=index_params)


In [ ]:
# we can run a check to see the collections in our milvus instance and we see the new collection has been created 

from pymilvus import utility
utility.list_collections()

In [ ]:
# load data into Milvus
import pandas as pd
from sentence_transformers import SentenceTransformer
from pymilvus import Collection, connections
import warnings
warnings.filterwarnings('ignore')


for i in range(len(article_titles)):
    # Create vector embeddings + data
    model = SentenceTransformer('sentence-transformers/all-minilm-l12-v2') # 384 dim
    passage_embeddings = model.encode(article_chunks[i])

    basic_collection = Collection(collection_name) 
    data = [
        article_chunks[i],
        article_titles[i],
        passage_embeddings
    ]
               
    out = basic_collection.insert(data)
    basic_collection.flush()  # Ensures data persistence

    
    print("Wikipedia Article: \'" + article_titles[i][0] + "\' has been loaded.")

In [ ]:
## check to ensure entities have been loaded into the collection

basic_collection = Collection(collection_name) 

basic_collection.num_entities

# Step 4: Searching with Milvus

In [ ]:
from sentence_transformers import SentenceTransformer
from pymilvus import(
    Milvus,
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema,
)

url = milvus_credentials['host']
port = milvus_credentials['port']
apikey = milvus_credentials['password']
apiuser = 'ibmlhapikey'


connections.connect(alias="default", 
                    host=url, 
                    port=port, 
                    user=apiuser, 
                    password=apikey, 
                    secure=True)


# Load collection

basic_collection = Collection(collection_name)      
basic_collection.load()

# Query function
def query_milvus(query, num_results):
    
    # Vectorize query
    model = SentenceTransformer('sentence-transformers/all-minilm-l12-v2') # 384 dim
    query_embeddings = model.encode([query])

    # Search
    search_params = {
        "metric_type": "L2", 
        "params": {"nprobe": 5}
    }
    results = basic_collection.search(
        data=query_embeddings, 
        anns_field="vector", 
        param=search_params,
        limit=num_results,
        expr=None, 
        output_fields=['article_text'],
    )
    return results

# Prompt with LLM

In [ ]:
## Consider some questions to ask regarding the topic you have chosen 

#question_text = "OTHER QUESTION TEXT"

question_text = "How to add a new catalog?"

In [ ]:
# Query Milvus 

num_results = 3
results = query_milvus(question_text, num_results)

relevant_chunks = []
for i in range(num_results):    
    #print(f"id: {results[0].ids[i]}")
    #print(f"distance: {results[0].distances[i]}")
    text = results[0][i].entity.get('article_text')
    relevant_chunks.append(text)
    
#print(relevant_chunks)

In [ ]:
def make_prompt(context, question_text):
    return (f"{context}\n\nPlease answer a question using this text. "
          + f"If the question is unanswerable, say \"unanswerable\"."
          + f"\n\nQuestion: {question_text}")


# Build prompt w/ Milvus results
# Embed retrieved passages(context) and user question into into prompt text

context = "\n\n".join(relevant_chunks)
prompt = make_prompt(context, question_text)

print(prompt)

In [ ]:
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

# Model Parameters
params = {
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.MAX_NEW_TOKENS: 500,
        GenParams.TEMPERATURE: 0,
}


# please note if using a cloud account in a different geography the cloud URL will be different 
# Refer to this list: 
#    Dallas - https://us-south.ml.cloud.ibm.com
#    London - https://eu-gb.ml.cloud.ibm.com
#    Frankfurt - https://eu-de.ml.cloud.ibm.com
#    Tokyo - https://jp-tok.ml.cloud.ibm.com

creds = {
    "url": 'https://us-south.ml.cloud.ibm.com',
    "apikey": milvus_credentials['password'] 
}

model = Model(
        model_id='ibm/granite-13b-chat-v2', 
        #model_id='meta-llama/llama-2-70b-chat', 
        params=params, credentials=creds, 
        project_id=wslib.here.get_ID()
)

# Prompt LLM
response = model.generate_text(prompt)
print(f"Question: {question_text}{response}")